# Introduction

Once we finished preparing our crystal system, we can finally start the production runs. We want to clarify our goals here. The goal is to use MD simulation with a sufficiently good protein forcefield to sample protein configurations near the experimentally observed structure. Then by the ergodicity hypothesis we shall take snapshots over the trajectory to represent the ensemble of unit cell structure we would from the actual crystal from which we collect the diffraction data. To make clear, the structure obtained from diffraction experiment is a time average over the duration of X-ray illumination and also a spatial average over all unit cells in the crystal. Here, because the computational power is pretty cheap, we are hoping that the time averaged structure from a sufficiently long MD simulation of the crystal is comparable to the experimental structure. 

We note a caveat is that we are just simulating a 1x1x1 unit cell here, so with the periodic boundary condition, we introduce artificial periodicity in the crystal system represented by our simulation. This may lead to subtle artefacts like relative shifts of average positions of individual proteins in the unit cell system and also slight distortions of average protein structures that cannot be easily eliminated by superposition (alignment). And my experience is that these effects appear early won't go away unless we simulate for very long time, i.e., we got stuck in some metastable crystal conformation. So the strategy here is to run many replicates under the same simulation settings and using the same starting structure from the volume equilibration (squeeze) step just before we remove the constraints. In the analysis step I will show how the statistics from averaging over the replicates are actually useful.

Finally, 